<a href="https://colab.research.google.com/github/aksharat/Medical-Projects/blob/main/WarmUp/WarmUp_Exercise6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import random
import matplotlib.pyplot as plt
import numpy as np
from torchvision.models.detection import maskrcnn_resnet50_fpn, MaskRCNN_ResNet50_FPN_Weights
from torchvision import transforms, datasets



In [ ]:
!wget http://imgcom.jsrt.or.jp/imgcom/wp-content/uploads/2019/07/segmentation02.zip

--2023-09-23 18:17:03--  http://imgcom.jsrt.or.jp/imgcom/wp-content/uploads/2019/07/segmentation02.zip
Resolving imgcom.jsrt.or.jp (imgcom.jsrt.or.jp)... 158.199.228.161
Connecting to imgcom.jsrt.or.jp (imgcom.jsrt.or.jp)|158.199.228.161|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23368008 (22M) [application/zip]
Saving to: ‘segmentation02.zip’

segmentation02.zip  100%[===================>]  22.29M   974KB/s    in 31s     

2023-09-23 18:17:34 (737 KB/s) - ‘segmentation02.zip’ saved [23368008/23368008]



In [ ]:
!unzip /content/segmentation02.zip

In [ ]:
weights = MaskRCNN_ResNet50_FPN_Weights.DEFAULT
model = maskrcnn_resnet50_fpn(weights=weights)
# transforms = weights.transforms()

In [ ]:
import os
import cv2
import numpy as np
import json

def detect_all_classes_bounding_boxes_and_save(label_dir, output_dir):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate through all label images in the label_dir
    for filename in os.listdir(label_dir):
        if filename.endswith("_label.png"):
            label_image_path = os.path.join(label_dir, filename)

            # Define pixel value ranges for each class
            class_pixel_ranges = [
                ((255, 255, 255), (255, 255, 255), (255, 255, 255)),  # Lung field
                ((85, 85, 85), (85, 85, 85), (85, 85, 85)),  # Heart region
                ((170, 170, 170), (170, 170, 170), (170, 170, 170)),  # Another Lung field
                ((0, 0, 0), (0, 0, 0), (0, 0, 0))  # Outside the body
            ]

            # Load the label image (teacher image)
            label_image = cv2.imread(label_image_path)

            # Initialize a dictionary to store bounding boxes for each class
            bounding_boxes = {i: [] for i in range(len(class_pixel_ranges))}

            # Create masks for each class and find contours
            for class_idx, pixel_range in enumerate(class_pixel_ranges):
                masks = [cv2.inRange(label_image, np.array(range), np.array(range)) for range in pixel_range]
                for mask in masks:
                    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    for contour in contours:
                        x, y, w, h = cv2.boundingRect(contour)

                        # Check if the bounding box is valid
                        if x >= 0 and y >= 0 and w > 0 and h > 0:
                            bounding_boxes[class_idx].append({"x": x, "y": y, "width": w, "height": h})

            # Combine bounding boxes for "chest area" (excluding outside and heart)
            chest_area_bounding_boxes = []
            for i in range(len(class_pixel_ranges)):
                if i not in [0, 3, 1]:
                    chest_area_bounding_boxes += bounding_boxes[i]

            # Save bounding box information to a JSON file
            case_name = os.path.splitext(filename)[0]  # Get the case name without extension
            output_json_path = os.path.join(output_dir, f"{case_name}.json")
            bounding_box_info = {
                "LungField": bounding_boxes[0],
                "HeartRegion": bounding_boxes[1],
                "AnotherLungField": bounding_boxes[2],
                "OutsideTheBody": bounding_boxes[3],
                "ChestArea": chest_area_bounding_boxes
            }

            with open(output_json_path, 'w') as json_file:
                json.dump(bounding_box_info, json_file, indent=2)  # Use indent for pretty formatting

            print(f"Saved bounding box information for {case_name} to {output_json_path}")

# Example usage:
train_label_dir = '/content/segmentation02/segmentation/label_train'
test_label_dir = '/content/segmentation02/segmentation/label_test'
train_output_dir = '/content/train_bbox_info'  # Modify this as needed
test_output_dir = '/content/test_bbox_info'  # Modify this as needed

detect_all_classes_bounding_boxes_and_save(train_label_dir, train_output_dir)
detect_all_classes_bounding_boxes_and_save(test_label_dir, test_output_dir)


In [ ]:
import os
import cv2
import numpy as np
import json
from PIL import Image
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, image_dir, json_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.json_dir = json_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_paths = os.listdir(image_dir)
        self.json_files = os.listdir(json_dir)
        self.mask_paths = os.listdir(mask_dir)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_paths[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_paths[idx].replace('.bmp', '_label.png'))
        json_file = os.path.join(self.json_dir, self.mask_paths[idx].replace('.png', '.json'))

        # Load image, mask, and JSON file
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Load JSON file and check for issues
        with open(json_file, 'r') as f:
            bbox_info = json.load(f)


        # Convert numpy arrays to PIL Images
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        mask = Image.fromarray(mask)

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)  # Apply the same transformation to the mask

        return image, mask, bbox_info


def check_bbox_issues(image, mask, bbox_info):
    issues = []

    for category, bboxes in bbox_info.items():
        for bbox in bboxes:
            x = bbox["x"]
            y = bbox["y"]
            width = bbox["width"]
            height = bbox["height"]

            if x < 0 or y < 0 or x + width > image.shape[1] or y + height > image.shape[0]:
                issues.append(f"Invalid bbox in category '{category}': {bbox}")

    return issues




In [ ]:
# Example usage:
train_image_dir = '/content/segmentation02/segmentation/org_train'
train_json_dir = '/content/train_bbox_info'
train_mask_dir = '/content/segmentation02/segmentation/label_train'

# Define image transformations if needed (e.g., resizing, normalization)
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

# Create a custom dataset instance for training
train_dataset = CustomDataset(train_image_dir, train_json_dir, train_mask_dir, transform)


In [ ]:
# Example usage:
test_image_dir = '/content/segmentation02/segmentation/org_test'
test_json_dir = '/content/test_bbox_info'
test_mask_dir = '/content/segmentation02/segmentation/label_test'


# Create a custom dataset instance for training
test_dataset = CustomDataset(test_image_dir, test_json_dir, test_mask_dir, transform)


In [ ]:
import torch
from torch.utils.data import DataLoader

# Example usage:
train_batch_size = 1  # You can adjust this batch size as needed
test_batch_size = 1   # You can adjust this batch size as needed

# Create DataLoader instances with the custom collate function
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)



In [ ]:
# Access properties of the train_loader
print(f"Train DataLoader Properties:")
print(f"Batch Size: {train_loader.batch_size}")
print(f"Number of Workers: {train_loader.num_workers}")
print(f"Drop Last Batch: {train_loader.drop_last}")
print(f"Dataset Length: {len(train_loader.dataset)}")

# Access properties of the test_loader
print(f"\nTest DataLoader Properties:")
print(f"Batch Size: {test_loader.batch_size}")
print(f"Number of Workers: {test_loader.num_workers}")
print(f"Drop Last Batch: {test_loader.drop_last}")
print(f"Dataset Length: {len(test_loader.dataset)}")


Train DataLoader Properties:
Batch Size: 1
Number of Workers: 0
Drop Last Batch: False
Dataset Length: 199

Test DataLoader Properties:
Batch Size: 1
Number of Workers: 0
Drop Last Batch: False
Dataset Length: 48


In [ ]:
import torchvision
# Initialize the Mask R-CNN model with ResNet-50 backbone
model = maskrcnn_resnet50_fpn(weights=weights)  # You can specify pretrained=True if you want pre-trained weights

# Modify the number of classes in the model
num_classes = 5  # Replace with the number of classes in your dataset
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)


# Set the device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:

# Define loss function and optimizer (replace with your own)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10  # Replace with the number of training epochs you want
train_losses = []
test_losses = []

In [ ]:
import torchvision.transforms as T

In [ ]:
# Assuming you have already created your train_loader
print(train_loader)


In [ ]:

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for batch_idx, (images, targets,bbox) in enumerate(train_loader):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()

        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backward pass and optimization
        losses.backward()
        optimizer.step()

        train_loss += losses.item()

    # Calculate average training loss for the epoch
    train_loss /= len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f}')


AttributeError: ignored

In [ ]:
num_batches = len(train_loader)
print(f"Total number of batches in train_loader: {num_batches}")


Total number of batches in train_loader: 199


In [ ]:
import os
import torch
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image

# Directory containing the .bmp images
image_dir = "/content/segmentation02/segmentation/org_train_s/"

# List all the .bmp image files in the directory
image_files = [f for f in os.listdir(image_dir) if f.endswith(".bmp")]

# Define the transformation to apply to each image
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to a PyTorch tensor
])

# Load and preprocess each image
images = []
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    image = Image.open(image_path).convert("RGB")  # Open the .bmp image and convert to RGB mode
    image = transform(image)  # Apply the defined transformation
    images.append(image)

# Create the Faster R-CNN model with default weights
model = fasterrcnn_resnet50_fpn(pretrained=True, progress=True)
model = model.eval()

# Perform inference on the preprocessed images
with torch.no_grad():
    outputs = model(images)

# outputs now contains the detection results for each input image
print(outputs)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[{'boxes': tensor([[ 10.5409,  18.1177, 249.7619, 248.6298],
        [ 16.5886,  19.8839, 220.0952, 242.7390],
        [  0.3504,  16.9393, 255.5071, 241.3622],
        [135.5676,  24.6512, 245.1338, 224.5948],
        [  5.0574,  26.8288, 140.5573, 232.9227],
        [  8.3246,  21.7068, 124.5667, 215.0065],
        [ 81.2057,  26.9080, 214.6474, 203.7318],
        [131.5770,  17.5968, 250.8983, 246.8999],
        [  1.5200,  11.9907, 253.7998, 253.0857],
        [ 80.0196,  27.9006, 196.3253, 198.9382],
        [  0.0000,  19.2021, 255.2403, 240.8332],
        [161.8391,  16.6125, 251.6003, 243.8387],
        [121.2233,  25.4828, 252.5641, 248.5527],
        [107.5567,  22.0460, 248.9638, 243.1363],
        [  8.3708,  18.1344, 126.0205, 217.8614],
        [ 18.0580,  14.0012, 121.3181, 223.4219],
        [ 24.8805,  23.9808, 226.5685, 232.8967]]), 'labels': tensor([ 1, 17, 63,  1,  1, 63,  1, 63, 62, 17, 65, 65, 62, 32, 65, 62, 18]), 'scores': tensor([0.5965, 0.4321, 0.3697, 0.3426,